In [1]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlparse, parse_qs, unquote, urlencode
import re
import json

In [ ]:
POSTHOG_PERSONAL_API_KEY = "XXXXX"
PROJECT_ID = "39308"
BASE_URL = "https://eu.posthog.com"

# Chargement des données via API


In [3]:
url = f"https://eu.posthog.com/api/projects/{PROJECT_ID}/query/"
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {POSTHOG_PERSONAL_API_KEY}'
}
payload = {
    "query": {
        "kind": "HogQLQuery",
        "query": """
        select timestamp, properties, distinct_id, `$session_id`, properties.$current_url, person_id
from events
where
and(
    event = '$pageview',
    notIn(
        properties.$host, tuple(
            'facili-tacct-preprod.osc-fr1.scalingo.io',
            'facili-tacct.osc-secnum-fr1.scalingo.io',
            'facili-tacct-nginx.osc-secnum-fr1.scalingo.io',
            'facili-tacct.incubateur.ademe.dev',
            'null'
        )
    ),
)
AND timestamp < now()
AND properties.$current_url NOT LIKE '%localhost%'
AND properties.$current_url LIKE 'https://facili-tacct.beta.gouv.fr/%'
AND properties.$current_url NOT LIKE '%scalingo%'
AND properties.$current_url NOT LIKE '%secnum%'
order by timestamp desc
limit 100000
        """
    }
}
response = requests.post(url, headers=headers, data=json.dumps(payload))
df = response.json()
df_north_star = pd.DataFrame(df.get("results"))
export = df_north_star.rename({
    0: "timestamp",
    1: "properties",
    2: "distinct_id",
    3: "session_id",
    4: "url",
    5: "person_id"
}, axis=1)
export

,timestamp,properties,distinct_id,session_id,url,person_id
0,2025-09-23T09:06:16.531000+02:00,"{""$prev_pageview_last_content_percentage"":0.19...",01997564-d92f-7bdd-9809-34aff42e2bba,01997564-d92e-76eb-aff8-e0dca461393b,https://facili-tacct.beta.gouv.fr/thematiques?...,a613ba3f-e511-5de2-bb7c-ce709b99661e
1,2025-09-23T09:06:07.194000+02:00,"{""$referrer"":""https://facili-tacct.beta.gouv.f...",01997564-d92f-7bdd-9809-34aff42e2bba,01997564-d92e-76eb-aff8-e0dca461393b,https://facili-tacct.beta.gouv.fr/,a613ba3f-e511-5de2-bb7c-ce709b99661e
2,2025-09-23T09:05:38.046000+02:00,"{""$host"":""facili-tacct.beta.gouv.fr"",""$raw_use...",01997564-66ff-73fd-aae2-a3ca50fda6ec,01997564-66ff-73fd-aae2-a3c802de93d0,https://facili-tacct.beta.gouv.fr/,53dd7500-81af-5a28-8159-7920824c77c2
3,2025-09-23T07:47:40.414000+02:00,"{""$session_id"":""0199751d-04d3-7710-a77b-3a2f73...",0199751d-04d6-7bcd-b87e-58908b2f37b3,0199751d-04d3-7710-a77b-3a2f73b51b9d,https://facili-tacct.beta.gouv.fr/donnees-terr...,359de009-f631-51fd-82db-6a41012638a6
4,2025-09-22T18:21:30.689000+02:00,"{""$referring_domain"":""$direct"",""$replay_sample...",0199723a-f400-7caa-a5f0-03fc72992d02,0199723a-f3fe-7ef8-98ec-8cadeb6ce063,https://facili-tacct.beta.gouv.fr/,f04f5288-0d20-5b44-aa83-1a9cf3acb173
...,...,...,...,...,...,...
34411,2024-12-04T12:04:22.687000+01:00,"{""$os"":""Windows"",""$os_version"":""10"",""$browser""...",01939157-ad5a-75dc-bf85-a9fa71972090,01939157-ad5a-75dc-bf85-a9f85475ace0,https://facili-tacct.beta.gouv.fr/,49fbaf96-3e1b-5382-9ee2-80e68e6e06ee
34412,2024-12-04T11:56:14.169000+01:00,"{""$prev_pageview_pathname"":""/thematiques"",""tit...",01939150-07f5-77bc-8f8b-36d1bdbd040f,01939150-07f4-7b89-be09-477ac7b8d5e4,https://facili-tacct.beta.gouv.fr/donnees-terr...,aae7dfb2-68d2-55ed-806f-5bb2a8418d3a
34413,2024-12-04T11:56:11.194000+01:00,"{""$time"":1733309780.594,""$pathname"":""/thematiq...",01939150-07f5-77bc-8f8b-36d1bdbd040f,01939150-07f4-7b89-be09-477ac7b8d5e4,https://facili-tacct.beta.gouv.fr/thematiques?...,aae7dfb2-68d2-55ed-806f-5bb2a8418d3a
34414,2024-12-04T11:55:52.172000+01:00,"{""$os"":""Windows"",""$os_version"":""10"",""$browser""...",01939150-07f5-77bc-8f8b-36d1bdbd040f,01939150-07f4-7b89-be09-477ac7b8d5e4,https://facili-tacct.beta.gouv.fr/,aae7dfb2-68d2-55ed-806f-5bb2a8418d3a


In [86]:
export

,timestamp,properties,distinct_id,session_id,url,person_id
0,2025-09-22T18:21:30.689000+02:00,"{""$referring_domain"":""$direct"",""$replay_sample...",0199723a-f400-7caa-a5f0-03fc72992d02,0199723a-f3fe-7ef8-98ec-8cadeb6ce063,https://facili-tacct.beta.gouv.fr/,f04f5288-0d20-5b44-aa83-1a9cf3acb173
1,2025-09-22T17:59:16.907000+02:00,"{""$screen_width"":1680,""$lib_version"":""1.214.1""...",019970a2-9a8a-72a6-9bf5-4cb38313ee7a,019971f9-205a-7d6c-ba74-d5266dee2d20,https://facili-tacct.beta.gouv.fr/donnees-terr...,1ddf2576-cc2d-5bbe-814b-0391df359fa2
2,2025-09-22T17:45:26.077000+02:00,"{""$session_recording_start_reason"":""session_id...",019970a2-9a8a-72a6-9bf5-4cb38313ee7a,019971f9-205a-7d6c-ba74-d5266dee2d20,https://facili-tacct.beta.gouv.fr/thematiques?...,1ddf2576-cc2d-5bbe-814b-0391df359fa2
3,2025-09-22T17:45:23.995000+02:00,"{""$console_log_recording_enabled_server_side"":...",019970a2-9a8a-72a6-9bf5-4cb38313ee7a,019971f9-205a-7d6c-ba74-d5266dee2d20,https://facili-tacct.beta.gouv.fr/donnees-terr...,1ddf2576-cc2d-5bbe-814b-0391df359fa2
4,2025-09-22T17:45:23.245000+02:00,"{""$session_id"":""019971f9-205a-7d6c-ba74-d5266d...",019970a2-9a8a-72a6-9bf5-4cb38313ee7a,019971f9-205a-7d6c-ba74-d5266dee2d20,https://facili-tacct.beta.gouv.fr/thematiques?...,1ddf2576-cc2d-5bbe-814b-0391df359fa2
...,...,...,...,...,...,...
34407,2024-12-04T12:04:22.687000+01:00,"{""$os"":""Windows"",""$os_version"":""10"",""$browser""...",01939157-ad5a-75dc-bf85-a9fa71972090,01939157-ad5a-75dc-bf85-a9f85475ace0,https://facili-tacct.beta.gouv.fr/,49fbaf96-3e1b-5382-9ee2-80e68e6e06ee
34408,2024-12-04T11:56:14.169000+01:00,"{""$prev_pageview_pathname"":""/thematiques"",""tit...",01939150-07f5-77bc-8f8b-36d1bdbd040f,01939150-07f4-7b89-be09-477ac7b8d5e4,https://facili-tacct.beta.gouv.fr/donnees-terr...,aae7dfb2-68d2-55ed-806f-5bb2a8418d3a
34409,2024-12-04T11:56:11.194000+01:00,"{""$time"":1733309780.594,""$pathname"":""/thematiq...",01939150-07f5-77bc-8f8b-36d1bdbd040f,01939150-07f4-7b89-be09-477ac7b8d5e4,https://facili-tacct.beta.gouv.fr/thematiques?...,aae7dfb2-68d2-55ed-806f-5bb2a8418d3a
34410,2024-12-04T11:55:52.172000+01:00,"{""$os"":""Windows"",""$os_version"":""10"",""$browser""...",01939150-07f5-77bc-8f8b-36d1bdbd040f,01939150-07f4-7b89-be09-477ac7b8d5e4,https://facili-tacct.beta.gouv.fr/,aae7dfb2-68d2-55ed-806f-5bb2a8418d3a


# Extraction des données


## epci


In [10]:
import pandas as pd
import re

# Assumons que 'export' est votre DataFrame avec les colonnes timestamp, properties, distinct_id, session_id, url, person_id

# Filtrer les sessions qui contiennent à la fois /thematiques et /donnees-territoriales avec type=epci
sessions_with_both = export.groupby('session_id').filter(
    lambda df: df['url'].str.contains('thematiques', na=False).any() and 
               df['url'].str.contains('donnees-territoriales', na=False).any() and
               df['url'].str.contains('type=epci', na=False).any()
)

print(f"Nombre de sessions filtrées (avec thematiques, donnees, type=epci) : {len(sessions_with_both['session_id'].unique())}")

# Maintenant, pour chaque session valide, extraire les codes et thématiques
code_summary = {}
total_codes_recherches = 0
total_codes_consultes = 0

for session_id in sessions_with_both['session_id'].unique():
    session_df = sessions_with_both[sessions_with_both['session_id'] == session_id]
    
    # URLs thematiques avec type=epci
    thematiques_urls = session_df[session_df['url'].str.contains('thematiques', na=False) & 
                                  session_df['url'].str.contains('type=epci', na=False)]
    
    # Extraire les codes des URLs thematiques
    codes_recherches = set()  # Utiliser un set pour unicité
    for url in thematiques_urls['url']:
        match = re.search(r'([0-9]{9})', url)  # Pour epci, code à 9 chiffres
        if match:
            codes_recherches.add(match.group())
    
    total_codes_recherches += len(codes_recherches)
    
    # URLs donnees-territoriales avec type=epci
    donnees_urls = session_df[session_df['url'].str.contains('donnees-territoriales', na=False) & 
                              session_df['url'].str.contains('type=epci', na=False)]
    
    # Collecter les codes consultés dans cette session (unicité)
    codes_consultes = set()
    for url in donnees_urls['url']:
        match_code = re.search(r'([0-9]{9})', url)
        if match_code:
            code = match_code.group()
            if code in codes_recherches:
                thematique = url.split('thematique=', 1)[1] if 'thematique=' in url else None
                if thematique:
                    codes_consultes.add(code)  # Ajouter au set pour unicité par session
                    # Initialiser si nécessaire
                    if code not in code_summary:
                        code_summary[code] = {"recherche_count": 0, "thematiques": set()}
                    code_summary[code]["thematiques"].add(thematique.replace("+", "_"))
    
    total_codes_consultes += len(codes_consultes)
    
    # Incrémenter recherche_count une fois par code consulté dans cette session
    for code in codes_consultes:
        code_summary[code]["recherche_count"] += 1

print(f"Total codes recherchés (unique par session) : {total_codes_recherches}")
print(f"Total codes consultés (unique par session) : {total_codes_consultes}")
print(f"Nombre de codes uniques dans code_summary : {len(code_summary)}")

# Maintenant, filtrer les codes avec >=3 recherches et >=3 thématiques distinctes
filtered_codes = [
    code for code, info in code_summary.items()
    if info["recherche_count"] >= 3 and len(info["thematiques"]) >= 3
]

print(f"Nombre de codes filtrés (>=3 recherches et >=3 thématiques) : {len(filtered_codes)}")

# La métrique : nombre de territoires (codes) qui satisfont les conditions
metrique = len(filtered_codes)

print(f"Nombre de territoires EPCI recherchés au moins 3 fois avec au moins 3 thématiques consultées : {metrique}")

Nombre de sessions filtrées (avec thematiques, donnees, type=epci) : 1157
Total codes recherchés (unique par session) : 1352
Total codes consultés (unique par session) : 1269
Nombre de codes uniques dans code_summary : 317
Nombre de codes filtrés (>=3 recherches et >=3 thématiques) : 125
Nombre de territoires EPCI recherchés au moins 3 fois avec au moins 3 thématiques consultées : 125


In [9]:
import pandas as pd
import re

# Assumons que 'export' est votre DataFrame avec les colonnes timestamp, properties, distinct_id, session_id, url, person_id

# Filtrer les sessions qui contiennent à la fois /thematiques et /donnees-territoriales avec type=epci
sessions_with_both = export.groupby('session_id').filter(
    lambda df: df['url'].str.contains('thematiques', na=False).any() and 
               df['url'].str.contains('donnees-territoriales', na=False).any() and
               (df['url'].str.contains('codepci', na=False) & ~df['url'].str.contains('codgeo', na=False)).any()
)

print(f"Nombre de sessions filtrées (avec thematiques, donnees, type=epci) : {len(sessions_with_both['session_id'].unique())}")

# Maintenant, pour chaque session valide, extraire les codes et thématiques
code_summary = {}
total_codes_recherches = 0
total_codes_consultes = 0

for session_id in sessions_with_both['session_id'].unique():
    session_df = sessions_with_both[sessions_with_both['session_id'] == session_id]
    
    # URLs thematiques avec type=epci
    thematiques_urls = session_df[session_df['url'].str.contains('thematiques', na=False) & 
                                  session_df['url'].str.contains('codepci', na=False) & 
                                  ~session_df['url'].str.contains('codgeo', na=False)]
    
    # Extraire les codes des URLs thematiques
    codes_recherches = set()  # Utiliser un set pour unicité
    for url in thematiques_urls['url']:
        match = re.search(r'([0-9]{9})', url)  # Pour epci, code à 9 chiffres
        if match:
            codes_recherches.add(match.group())
    
    total_codes_recherches += len(codes_recherches)
    
    # URLs donnees-territoriales avec type=epci
    donnees_urls = session_df[session_df['url'].str.contains('donnees-territoriales', na=False) & 
                              session_df['url'].str.contains('codepci', na=False) & 
                                  ~session_df['url'].str.contains('codgeo', na=False)]
    
    # Collecter les codes consultés dans cette session (unicité)
    codes_consultes = set()
    for url in donnees_urls['url']:
        match_code = re.search(r'([0-9]{9})', url)
        if match_code:
            code = match_code.group()
            if code in codes_recherches:
                thematique = url.split('thematique=', 1)[1] if 'thematique=' in url else None
                if thematique:
                    codes_consultes.add(code)  # Ajouter au set pour unicité par session
                    # Initialiser si nécessaire
                    if code not in code_summary:
                        code_summary[code] = {"recherche_count": 0, "thematiques": set()}
                    code_summary[code]["thematiques"].add(thematique.replace("+", "_"))
    
    total_codes_consultes += len(codes_consultes)
    
    # Incrémenter recherche_count une fois par code consulté dans cette session
    for code in codes_consultes:
        code_summary[code]["recherche_count"] += 1

print(f"Total codes recherchés (unique par session) : {total_codes_recherches}")
print(f"Total codes consultés (unique par session) : {total_codes_consultes}")
print(f"Nombre de codes uniques dans code_summary : {len(code_summary)}")

# Maintenant, filtrer les codes avec >=3 recherches et >=3 thématiques distinctes
filtered_codes2 = [
    code for code, info in code_summary.items()
    if info["recherche_count"] >= 3 and len(info["thematiques"]) >= 3
]

print(f"Nombre de codes filtrés (>=3 recherches et >=3 thématiques) : {len(filtered_codes2)}")

# La métrique : nombre de territoires (codes) qui satisfont les conditions
metrique2 = len(filtered_codes2)

print(f"Nombre de territoires EPCI recherchés au moins 3 fois avec au moins 3 thématiques consultées : {metrique2}")

Nombre de sessions filtrées (avec thematiques, donnees, type=epci) : 781
Total codes recherchés (unique par session) : 866
Total codes consultés (unique par session) : 812
Nombre de codes uniques dans code_summary : 251
Nombre de codes filtrés (>=3 recherches et >=3 thématiques) : 76
Nombre de territoires EPCI recherchés au moins 3 fois avec au moins 3 thématiques consultées : 76


In [15]:
len(np.unique((filtered_codes + filtered_codes2)))

158

In [91]:
# Code pour les anciennes URLs uniquement (EPCI: codepci présent et codgeo absent)

import pandas as pd
import re
from urllib.parse import urlparse, parse_qs

# Filtrer les sessions avec anciennes URLs
sessions_old = export.groupby('session_id').filter(
    lambda df: df['url'].str.contains('thematiques', na=False).any() and 
               df['url'].str.contains('donnees-territoriales', na=False).any() and
               (df['url'].str.contains('codepci', na=False) & ~df['url'].str.contains('codgeo', na=False)).any()
)

print(f"Nombre de sessions filtrées (anciennes URLs) : {len(sessions_old['session_id'].unique())}")

# Extraire les codes et thématiques pour anciennes URLs
code_summary_old = {}
total_codes_recherches_old = 0
total_codes_consultes_old = 0

for session_id in sessions_old['session_id'].unique():
    session_df = sessions_old[sessions_old['session_id'] == session_id]
    
    # URLs thematiques avec anciennes conditions
    thematiques_urls = session_df[session_df['url'].str.contains('thematiques', na=False) & 
                                  session_df['url'].str.contains('codepci', na=False) & 
                                  ~session_df['url'].str.contains('codgeo', na=False)]
    
    # Extraire les codes des URLs thematiques (depuis params)
    codes_recherches = set()
    for url in thematiques_urls['url']:
        parsed_url = urlparse(url)
        params = parse_qs(parsed_url.query)
        code = params.get('codepci', [None])[0]
        if code:
            codes_recherches.add(code)
    
    total_codes_recherches_old += len(codes_recherches)
    
    # URLs donnees-territoriales avec anciennes conditions
    donnees_urls = session_df[session_df['url'].str.contains('donnees-territoriales', na=False) & 
                              session_df['url'].str.contains('codepci', na=False) & 
                              ~session_df['url'].str.contains('codgeo', na=False)]
    
    codes_consultes = set()
    for url in donnees_urls['url']:
        parsed_url = urlparse(url)
        params = parse_qs(parsed_url.query)
        code = params.get('codepci', [None])[0]
        if code and code in codes_recherches:
            thematique = url.split('thematique=', 1)[1] if 'thematique=' in url else None
            if thematique:
                codes_consultes.add(code)
                if code not in code_summary_old:
                    code_summary_old[code] = {"recherche_count": 0, "thematiques": set()}
                code_summary_old[code]["thematiques"].add(thematique.replace("+", "_"))
    
    total_codes_consultes_old += len(codes_consultes)
    
    for code in codes_consultes:
        code_summary_old[code]["recherche_count"] += 1

print(f"Total codes recherchés (anciennes URLs) : {total_codes_recherches_old}")
print(f"Total codes consultés (anciennes URLs) : {total_codes_consultes_old}")
print(f"Nombre de codes uniques (anciennes URLs) : {len(code_summary_old)}")

# Filtrer les codes avec >=3 recherches et >=3 thématiques
filtered_codes_old = [
    code for code, info in code_summary_old.items()
    if info["recherche_count"] >= 3 and len(info["thematiques"]) >= 3
]

print(f"Nombre de codes filtrés (anciennes URLs) : {len(filtered_codes_old)}")

# Métrique pour anciennes URLs
metrique_old = len(filtered_codes_old)
print(f"Nombre de territoires EPCI (anciennes URLs) : {metrique_old}")

Nombre de sessions filtrées (anciennes URLs) : 781
Total codes recherchés (anciennes URLs) : 866
Total codes consultés (anciennes URLs) : 812
Nombre de codes uniques (anciennes URLs) : 251
Nombre de codes filtrés (anciennes URLs) : 76
Nombre de territoires EPCI (anciennes URLs) : 76
Total codes recherchés (anciennes URLs) : 866
Total codes consultés (anciennes URLs) : 812
Nombre de codes uniques (anciennes URLs) : 251
Nombre de codes filtrés (anciennes URLs) : 76
Nombre de territoires EPCI (anciennes URLs) : 76


In [82]:
len(filtered_result_epci)

91

In [66]:
for count, i in enumerate(export["session_id"].unique()):
    print("Session", count, ":", i)

Session 0 : 019971d9-a55a-740e-abbc-9b48668bab64
Session 1 : 019971d8-8fd4-7ee7-ba76-5033b4a368e0
Session 2 : 019971b6-f93f-75ad-a1a7-e287e6445b22
Session 3 : 019971af-80ff-72b3-94b7-a686ef83a5f8
Session 4 : 019971b6-a3af-7e23-99de-b19cc87ad7b6
Session 5 : 0199719b-4486-715d-8c75-de63f96d1df7
Session 6 : 0199719b-7112-7cdd-b0db-4c8c09146d0b
Session 7 : 01997198-950b-7d73-a54e-9ee32252222e
Session 8 : 01997197-fa27-7d7d-997d-ad087759c946
Session 9 : 01997197-9bac-78ac-b443-e6673883002e
Session 10 : 0199715a-0df1-7db6-b13a-686fa8fbbae3
Session 11 : 01997157-ee57-7899-84e0-e28e3285afe8
Session 12 : 019970f1-6c6d-7c34-a578-15d5f766111a
Session 13 : 019970a2-9a8a-72a6-9bf5-4cb171899ad8
Session 14 : 019970d0-668a-7081-a798-2ac7a0499082
Session 15 : 019970c1-6174-7d04-a527-e550fdba178c
Session 16 : 019970b4-ca35-7141-940c-a263dc4f0f77
Session 17 : 019970b6-b4e0-74cd-b554-05d3aacad244
Session 18 : 0199709b-11fb-7a4d-ab5a-e46cd508c574
Session 19 : 019970b5-b697-7c67-92e0-131c560bd335
Session 20

In [68]:
test = export.loc[export["session_id"] == "019971d9-a55a-740e-abbc-9b48668bab64"]
test

,timestamp,properties,distinct_id,session_id,url,person_id
0,2025-09-22T16:39:14.652000+02:00,"{""$recording_status"":""active"",""$session_record...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2
1,2025-09-22T16:39:07.293000+02:00,"{""$pathname"":""/thematiques"",""$raw_user_agent"":...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/thematiques?...,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2
2,2025-09-22T16:37:27.875000+02:00,"{""$prev_pageview_max_content"":2544,""$screen_he...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/donnees-terr...,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2
3,2025-09-22T16:35:41.424000+02:00,"{""$replay_sample_rate"":null,""$browser_language...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/thematiques?...,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2


In [72]:
df_with_thematiques = test.loc[test["url"].str.contains("thematiques")]
df_with_donnees = test.loc[test["url"].str.contains("donnees-territoriales")]

In [74]:
df_with_thematiques

,timestamp,properties,distinct_id,session_id,url,person_id
1,2025-09-22T16:39:07.293000+02:00,"{""$pathname"":""/thematiques"",""$raw_user_agent"":...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/thematiques?...,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2
3,2025-09-22T16:35:41.424000+02:00,"{""$replay_sample_rate"":null,""$browser_language...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/thematiques?...,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2


In [73]:
df_with_donnees

,timestamp,properties,distinct_id,session_id,url,person_id
2,2025-09-22T16:37:27.875000+02:00,"{""$prev_pageview_max_content"":2544,""$screen_he...",01946a37-c8b4-74d5-87ae-2a8f7517c299,019971d9-a55a-740e-abbc-9b48668bab64,https://facili-tacct.beta.gouv.fr/donnees-terr...,ffb074ca-b133-55d7-a1fd-530ba9ebf1e2


In [75]:
data_epci

[{'index': 0,
  'codes_recherches': ['200042646'],
  'territoires': [],
  'thematiques': {'200042646': 'Ressources+en+eau'},
  'type': 'epci'},
 {'index': 3,
  'codes_recherches': ['246900575'],
  'territoires': [],
  'thematiques': {'246900575': 'Inconfort+thermique'},
  'type': 'epci'},
 {'index': 13,
  'codes_recherches': ['200066520', '246900575'],
  'territoires': [],
  'thematiques': {'246900575': 'Biodiversit%C3%A9',
   '200066520': 'Biodiversit%C3%A9'},
  'type': 'epci'},
 {'index': 14,
  'codes_recherches': ['246600423'],
  'territoires': [],
  'thematiques': {'246600423': 'Am%C3%A9nagement'},
  'type': 'epci'},
 {'index': 16,
  'codes_recherches': ['240700617'],
  'territoires': [],
  'thematiques': {'240700617': 'Inconfort+thermique'},
  'type': 'epci'},
 {'index': 17,
  'codes_recherches': ['200042646'],
  'territoires': [],
  'thematiques': {'200042646': 'Ressources+en+eau'},
  'type': 'epci'},
 {'index': 43,
  'codes_recherches': ['246800445', '246800494', '246800569'],
 

In [65]:
import re

thematiques = []
codes = []
data_epci = []

for count, i in enumerate(export["session_id"].unique()):
    temp_df = export.loc[export["session_id"] == i]

    contains_thematiques = temp_df['url'].str.contains('thematiques', na=False).any()
    contains_donnees = temp_df['url'].str.contains('donnees-territoriales', na=False).any()
    mask = (
        temp_df['url'].str.contains('type=epci', na=False)
        #   |
        # (
        #     temp_df['url'].str.contains('codepci', na=False) &
        #     ~temp_df['url'].str.contains('codgeo', na=False)
        # )
    )

    contains_epci = mask.any()
    result = contains_thematiques and contains_donnees and contains_epci

    if result:
        data_epci.append({
            "index": count,
            "codes_recherches": [],
            "territoires": [],
            "thematiques": {},
            "type": ""
        })

        df_with_thematiques = temp_df.loc[temp_df["url"].str.contains("thematiques")]
        df_with_donnees = temp_df.loc[temp_df["url"].str.contains("donnees-territoriales")]
        codes_recherches = []

        for j in df_with_thematiques["url"]:
            if "type=epci" in j : #or ("codepci" in j and "codgeo" not in j)
                match = re.search(r'([0-9]{9})', j)
                if match:
                    code_thematiques = match.group()
                    codes_recherches.append(code_thematiques)
                else:
                    print(f"[DEBUG] Pas de code trouvé dans l’URL (thematique): {j}")
                libelle_territoire = j.split('thematique=', 1)[1] if 'thematique=' in j else None

        target_obj = next(obj for obj in data_epci if obj["index"] == count)
        target_obj["codes_recherches"].append(list(np.unique(codes_recherches)))
        target_obj["type"] = "epci"

        for k in df_with_donnees["url"]:
            if "type=epci" in k : #or ("codepci" in k and "codgeo" not in k)
                match = re.search(r'([0-9]{9})', k)
                if match:
                    code_donnees = match.group()
                    codes.append(code_donnees)
                    if any(code_donnees in s for s in codes_recherches):
                        thematique_temp = k.split('thematique=', 1)[1] if 'thematique=' in k else None
                        thematiques.append(thematique_temp)
                        target_obj2 = next(obj for obj in data_epci if obj["index"] == count)
                        target_obj2["thematiques"][code_donnees] = thematique_temp
                else:
                    print(f"[DEBUG] Pas de code trouvé dans l’URL (donnees): {k}")


for obj in data_epci:
    codes = obj["codes_recherches"]
    # Flatten only if it's a list with one element that is also a list
    if len(codes) == 1 and isinstance(codes[0], list):
        obj["codes_recherches"] = codes[0]

# Create a dictionary to collect data_epci_v2 per code
code_summary = {}

# Loop through each object
for obj in data_epci:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_epci = []

for code, info in code_summary.items():
    final_result_epci.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "epci"
    })

filtered_result_epci = [
    item for item in final_result_epci
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]

len(filtered_result_epci)

[DEBUG] Pas de code trouvé dans l’URL (thematique): https://facili-tacct.beta.gouv.fr/thematiques?code=01004&libelle=Amb%C3%A9rieu-en-Bugey&type=epci
[DEBUG] Pas de code trouvé dans l’URL (thematique): https://facili-tacct.beta.gouv.fr/thematiques?code=01004&libelle=Amb%C3%A9rieu-en-Bugey&type=epci
[DEBUG] Pas de code trouvé dans l’URL (thematique): https://facili-tacct.beta.gouv.fr/thematiques?code=01004&libelle=Amb%C3%A9rieu-en-Bugey&type=epci
[DEBUG] Pas de code trouvé dans l’URL (thematique): https://facili-tacct.beta.gouv.fr/thematiques?code=01004&libelle=Amb%C3%A9rieu-en-Bugey&type=epci
[DEBUG] Pas de code trouvé dans l’URL (thematique): https://facili-tacct.beta.gouv.fr/thematiques?code=01004&libelle=Amb%C3%A9rieu-en-Bugey&type=epci
[DEBUG] Pas de code trouvé dans l’URL (donnees): https://facili-tacct.beta.gouv.fr/donnees-territoriales?code=01004&libelle=Amb%C3%A9rieu-en-Bugey&type=epci&thematique=Ressources+en+eau
[DEBUG] Pas de code trouvé dans l’URL (donnees): https://facili-

91

In [46]:
# Create a dictionary to collect data_epci_v2 per code
code_summary = {}

# Loop through each object
for obj in data_epci:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_epci = []

for code, info in code_summary.items():
    final_result_epci.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "epci"
    })

filtered_result_epci = [
    item for item in final_result_epci
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]
filtered_result_epci

[{'code': '246900575',
  'recherche_count': 17,
  'thematiques_count': 6,
  'thematiques': ['Inconfort_thermique',
   'Agriculture',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Ressources_en_eau',
   'Gestion_des_risques'],
  'type': 'epci'},
 {'code': '240700617',
  'recherche_count': 4,
  'thematiques_count': 3,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Am%C3%A9nagement'],
  'type': 'epci'},
 {'code': '200046977',
  'recherche_count': 10,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Gestion_des_risques'],
  'type': 'epci'},
 {'code': '200071900',
  'recherche_count': 30,
  'thematiques_count': 6,
  'thematiques': ['Inconfort_thermique',
   'Agriculture',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Ressources_en_eau',
   'Gestion_des_risques'],
  'type': 'epci'},
 {'code': '200070720',
  'recherche_count': 4,
  'thematiques_count': 3,
  'thematiques': ['Inconfort_thermique

In [47]:
len(final_result_epci)

317

In [48]:
len(filtered_result_epci)

91

## Communes


In [15]:
%%time
thematiques = []
codes = []
data_commune = []

for count, i in enumerate(export["session_id"].unique()):
    temp_df = export.loc[export["session_id"] == i]

    contains_thematiques = temp_df['url'].str.contains('thematiques', na=False).any()
    contains_donnees = temp_df['url'].str.contains('donnees-territoriales', na=False).any()
    mask = (
        temp_df['url'].str.contains('type=commune', na=False) |
        (
            temp_df['url'].str.contains('codepci', na=False) &
            ~temp_df['url'].str.contains('codgeo', na=False)
        )
    )

    contains_commune = mask.any()
    result = contains_thematiques and contains_donnees and contains_commune

    if result:
        data_commune.append({
            "index": count,
            "codes_recherches": [],
            "thematiques": {},
            "type": "commune"
        })

        df_with_thematiques = temp_df.loc[temp_df["url"].str.contains("thematiques")]
        df_with_donnees = temp_df.loc[temp_df["url"].str.contains("donnees-territoriales")]
        codes_recherches = []

        for j in df_with_thematiques["url"]:
            if "type=commune" in j or ("codepci" in j and "codgeo" in j):
                # match = re.search(r'([0-9]{5})', j)
                match = re.search(r'(?:\d{5}|2A\d{3}|2B\d{3})', j)
                if match:
                    code_thematiques = match.group()
                    codes_recherches.append(code_thematiques)
                else:
                    print(f"[DEBUG] Aucun code commune trouvé dans l’URL (thematique): {j}")
                libelle_territoire = j.split('thematique=', 1)[1] if 'thematique=' in j else None

        target_obj = next(obj for obj in data_commune if obj["index"] == count)
        target_obj["codes_recherches"].append(list(np.unique(codes_recherches)))
        target_obj["type"] = "commune"

        for k in df_with_donnees["url"]:
            if "type=commune" in k or ("codepci" in k and "codgeo" in k):
                # match = re.search(r'([0-9]{5})', k)
                match = re.search(r'(?:\d{5}|2A\d{3}|2B\d{3})', k)
                if match:
                    code_donnees = match.group()
                    codes.append(code_donnees)
                    if any(code_donnees in s for s in codes_recherches):
                        thematique_temp = k.split('thematique=', 1)[1] if 'thematique=' in k else None
                        thematiques.append(thematique_temp)
                        target_obj2 = next(obj for obj in data_commune if obj["index"] == count)
                        target_obj2["thematiques"][code_donnees] = thematique_temp
                else:
                    print(f"[DEBUG] Aucun code commune trouvé dans l’URL (donnees): {k}")


CPU times: total: 37.1 s
Wall time: 37.5 s


In [16]:
# Create a dictionary to collect data_epci_v2 per code
code_summary = {}

# Loop through each object
for obj in data_commune:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_commune = []

for code, info in code_summary.items():
    final_result_commune.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "commune"
    })

filtered_result_commune = [
    item for item in final_result_commune
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]
filtered_result_commune

[{'code': '01053',
  'recherche_count': 3,
  'thematiques_count': 3,
  'thematiques': ['Ressources_en_eau',
   'Inconfort_thermique',
   'Gestion_des_risques'],
  'type': 'commune'},
 {'code': '17300',
  'recherche_count': 6,
  'thematiques_count': 3,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Gestion_des_risques'],
  'type': 'commune'},
 {'code': '42168',
  'recherche_count': 6,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Biodiversit%C3%A9',
   'Gestion_des_risques'],
  'type': 'commune'},
 {'code': '01004',
  'recherche_count': 28,
  'thematiques_count': 5,
  'thematiques': ['Inconfort_thermique',
   'Agriculture',
   'Biodiversit%C3%A9',
   'Ressources_en_eau',
   'Gestion_des_risques'],
  'type': 'commune'},
 {'code': '44109',
  'recherche_count': 10,
  'thematiques_count': 5,
  'thematiques': ['Inconfort_thermique',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Ressources_en_eau',
   'Gestion_des_ris

In [17]:
len(filtered_result_commune)

25

## PNR


In [18]:
%%time
thematiques = []
codes = []
data_pnr = []

for count, i in enumerate(export["session_id"].unique()):
    temp_df = export.loc[export["session_id"] == i]

    contains_thematiques = temp_df['url'].str.contains('thematiques', na=False).any()
    contains_donnees = temp_df['url'].str.contains('donnees-territoriales', na=False).any()
    contains_pnr = temp_df['url'].str.contains('type=pnr', na=False).any()

    result = contains_thematiques and contains_donnees and contains_pnr

    if result:
        data_pnr.append({
            "index": count,
            "codes_recherches": [],
            "thematiques": {},
            "type": "pnr"
        })

        df_with_thematiques = temp_df.loc[temp_df["url"].str.contains("thematiques")]
        df_with_donnees = temp_df.loc[temp_df["url"].str.contains("donnees-territoriales")]
        codes_recherches = []

        # Recherche des codes dans les URL "thematique"
        for j in df_with_thematiques["url"]:
            if "type=pnr" in j:
                match = re.search(r'(FR[0-9]{7})', j)
                if match:
                    code_thematiques = match.group()
                    codes_recherches.append(code_thematiques)
                else:
                    print(f"[DEBUG] Aucun code PNR trouvé dans l’URL (thematique): {j}")
                libelle_territoire = j.split('thematique=', 1)[1] if 'thematique=' in j else None

        target_obj = next(obj for obj in data_pnr if obj["index"] == count)
        target_obj["codes_recherches"].append(list(np.unique(codes_recherches)))
        target_obj["type"] = "pnr"

        # Recherche des codes dans les URL "donnees"
        for k in df_with_donnees["url"]:
            if "type=pnr" in k:
                match = re.search(r'(FR[0-9]{7})', k)
                if match:
                    code_donnees = match.group()
                    codes.append(code_donnees)
                    if any(code_donnees in s for s in codes_recherches):
                        thematique_temp = k.split('thematique=', 1)[1] if 'thematique=' in k else None
                        thematiques.append(thematique_temp)
                        target_obj2 = next(obj for obj in data_pnr if obj["index"] == count)
                        target_obj2["thematiques"][code_donnees] = thematique_temp
                else:
                    print(f"[DEBUG] Aucun code PNR trouvé dans l’URL (donnees): {k}")


CPU times: total: 32.7 s
Wall time: 33.2 s


In [19]:
code_summary = {}

# Loop through each object
for obj in data_pnr:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_pnr = []

for code, info in code_summary.items():
    final_result_pnr.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "pnr"
    })

filtered_result_pnr = [
    item for item in final_result_pnr
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]
filtered_result_pnr

[{'code': 'FR8000007',
  'recherche_count': 13,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Biodiversit%C3%A9',
   'Gestion_des_risques'],
  'type': 'pnr'},
 {'code': 'FR8000027',
  'recherche_count': 6,
  'thematiques_count': 4,
  'thematiques': ['Ressources_en_eau',
   'Inconfort_thermique',
   'Agriculture',
   'Gestion_des_risques'],
  'type': 'pnr'},
 {'code': 'FR8000015',
  'recherche_count': 4,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9'],
  'type': 'pnr'},
 {'code': 'FR8000035',
  'recherche_count': 20,
  'thematiques_count': 6,
  'thematiques': ['Inconfort_thermique',
   'Agriculture',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Ressources_en_eau',
   'Gestion_des_risques'],
  'type': 'pnr'}]

In [20]:
len(filtered_result_pnr)

4

## ept


In [21]:
%%time
thematiques = []
codes = []
data_ept = []

for count, i in enumerate(export["session_id"].unique()):
    temp_df = export.loc[export["session_id"] == i]

    contains_thematiques = temp_df['url'].str.contains('thematiques', na=False).any()
    contains_donnees = temp_df['url'].str.contains('donnees-territoriales', na=False).any()
    contains_ept = temp_df['url'].str.contains('type=ept', na=False).any()

    result = contains_thematiques and contains_donnees and contains_ept

    if result:
        data_ept.append({
            "index": count,
            "codes_recherches": [],
            "thematiques": {},
            "type": "ept"
        })

        df_with_thematiques = temp_df.loc[temp_df["url"].str.contains("thematiques")]
        df_with_donnees = temp_df.loc[temp_df["url"].str.contains("donnees-territoriales")]
        codes_recherches = []

        # Recherche dans les URL "thematique"
        for j in df_with_thematiques["url"]:
            if "type=ept" in j:
                match = re.search(r'T(?:[1-9]|1[0-2])', j)
                if match:
                    code_thematiques = match.group()
                    codes_recherches.append(code_thematiques)
                else:
                    print(f"[DEBUG] Aucun code EPT trouvé dans l’URL (thematique): {j}")
                libelle_territoire = j.split('thematique=', 1)[1] if 'thematique=' in j else None

        target_obj = next(obj for obj in data_ept if obj["index"] == count)
        target_obj["codes_recherches"].append(list(np.unique(codes_recherches)))
        target_obj["type"] = "ept"

        # Recherche dans les URL "donnees"
        for k in df_with_donnees["url"]:
            if "type=ept" in k:
                match = re.search(r'T(?:[1-9]|1[0-2])', k)
                if match:
                    code_donnees = match.group()
                    codes.append(code_donnees)
                    if any(code_donnees in s for s in codes_recherches):
                        thematique_temp = k.split('thematique=', 1)[1] if 'thematique=' in k else None
                        thematiques.append(thematique_temp)
                        target_obj2 = next(obj for obj in data_ept if obj["index"] == count)
                        target_obj2["thematiques"][code_donnees] = thematique_temp
                else:
                    print(f"[DEBUG] Aucun code EPT trouvé dans l’URL (donnees): {k}")


CPU times: total: 34.3 s
Wall time: 35.2 s


In [22]:
code_summary = {}

# Loop through each object
for obj in data_ept:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_ept = []

for code, info in code_summary.items():
    final_result_ept.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "ept"
    })

filtered_result_ept = [
    item for item in final_result_ept
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]
filtered_result_ept

[]

In [23]:
len(filtered_result_ept)

0

## Département


In [24]:
%%time
thematiques = []
codes = []
data_departement = []

for count, i in enumerate(export["session_id"].unique()):
    temp_df = export.loc[export["session_id"] == i]

    contains_thematiques = temp_df['url'].str.contains('thematiques', na=False).any()
    contains_donnees = temp_df['url'].str.contains('donnees-territoriales', na=False).any()
    contains_departement = temp_df['url'].str.contains('type=departement', na=False).any()

    result = contains_thematiques and contains_donnees and contains_departement

    if result:
        data_departement.append({
            "index": count,
            "codes_recherches": [],
            "thematiques": {},
            "type": "departement"
        })

        df_with_thematiques = temp_df.loc[temp_df["url"].str.contains("thematiques")]
        df_with_donnees = temp_df.loc[temp_df["url"].str.contains("donnees-territoriales")]
        codes_recherches = []

        # Recherche dans les URL "thematique"
        for j in df_with_thematiques["url"]:
            if "type=departement" in j:
                match = re.search(r'(?:\d{2}|2A|2B|9\d{2})', j)
                if match:
                    code_thematiques = match.group()
                    codes_recherches.append(code_thematiques)
                else:
                    print(f"[DEBUG] Aucun code département trouvé dans l’URL (thematique): {j}")
                libelle_territoire = j.split('thematique=', 1)[1] if 'thematique=' in j else None

        target_obj = next(obj for obj in data_departement if obj["index"] == count)
        target_obj["codes_recherches"].append(list(np.unique(codes_recherches)))
        target_obj["type"] = "departement"

        # Recherche dans les URL "donnees"
        for k in df_with_donnees["url"]:
            if "type=departement" in k:
                match = re.search(r'(?:\d{2}|2A|2B|9\d{2})', k)
                if match:
                    code_donnees = match.group()
                    codes.append(code_donnees)
                    if any(code_donnees in s for s in codes_recherches):
                        thematique_temp = k.split('thematique=', 1)[1] if 'thematique=' in k else None
                        thematiques.append(thematique_temp)
                        target_obj2 = next(obj for obj in data_departement if obj["index"] == count)
                        target_obj2["thematiques"][code_donnees] = thematique_temp
                else:
                    print(f"[DEBUG] Aucun code département trouvé dans l’URL (donnees): {k}")


CPU times: total: 33.3 s
Wall time: 34.4 s


In [25]:
code_summary = {}

# Loop through each object
for obj in data_departement:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_departement = []

for code, info in code_summary.items():
    final_result_departement.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "departement"
    })

filtered_result_departement = [
    item for item in final_result_departement
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]
filtered_result_departement

[{'code': '34',
  'recherche_count': 10,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Gestion_des_risques'],
  'type': 'departement'},
 {'code': '45',
  'recherche_count': 54,
  'thematiques_count': 6,
  'thematiques': ['Inconfort_thermique',
   'Agriculture',
   'Am%C3%A9nagement',
   'Biodiversit%C3%A9',
   'Ressources_en_eau',
   'Gestion_des_risques'],
  'type': 'departement'},
 {'code': '03',
  'recherche_count': 8,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Agriculture',
   'Gestion_des_risques'],
  'type': 'departement'},
 {'code': '06',
  'recherche_count': 5,
  'thematiques_count': 4,
  'thematiques': ['Ressources_en_eau',
   'Inconfort_thermique',
   'Agriculture',
   'Gestion_des_risques'],
  'type': 'departement'},
 {'code': '38',
  'recherche_count': 5,
  'thematiques_count': 4,
  'thematiques': ['Ressources_en_eau',
   'Inconfort_thermique',
   'Agr

In [26]:
len(filtered_result_departement)

28

## PETR


In [27]:
%%time
thematiques = []
codes = []
data_petr = []

for count, i in enumerate(export["session_id"].unique()):
    temp_df = export.loc[export["session_id"] == i]

    contains_thematiques = temp_df['url'].str.contains('thematiques', na=False).any()
    contains_donnees = temp_df['url'].str.contains('donnees-territoriales', na=False).any()
    contains_petr = temp_df['url'].str.contains('type=petr', na=False).any()

    result = contains_thematiques and contains_donnees and contains_petr

    if result:
        data_petr.append({
            "index": count,
            "codes_recherches": [],
            "thematiques": {},
            "type": "petr"
        })

        df_with_donnees = temp_df.loc[temp_df["url"].str.contains("donnees-territoriales")]
        petr_recherches = []

        for k in df_with_donnees["url"]:
            if "type=petr" in k:
                parsed_url = urlparse(k)
                params = parse_qs(parsed_url.query)

                # récupération du libellé
                libelle_encoded = params.get('libelle', [None])[0]
                if libelle_encoded:
                    libelle_petr = unquote(libelle_encoded.replace('+', ' '))
                    codes.append(libelle_petr)
                else:
                    libelle_petr = None
                    print(f"[DEBUG] Aucun libellé trouvé dans l’URL PETR: {k}")

                # récupération de la thématique
                thematique_temp = k.split('thematique=', 1)[1] if 'thematique=' in k else None
                if not thematique_temp:
                    print(f"[DEBUG] Aucune thématique trouvée dans l’URL PETR: {k}")

                thematiques.append(thematique_temp)

                target_obj2 = next(obj for obj in data_petr if obj["index"] == count)
                if libelle_petr:
                    target_obj2["thematiques"][libelle_petr] = thematique_temp


CPU times: total: 33.4 s
Wall time: 33.8 s


In [28]:
code_summary = {}

# Loop through each object
for obj in data_petr:
    thematiques = obj.get("thematiques", {})

    for code, label in thematiques.items():
        if code not in code_summary:
            code_summary[code] = {
                "code": code,
                "recherche_count": 0,
                "thematiques": set()
            }
        if label:
            code_summary[code]["recherche_count"] += 1
            code_summary[code]["thematiques"].add(label.replace("+", "_"))

# Build final array with thematiques_count and cleaned thematiques
final_result_petr = []

for code, info in code_summary.items():
    final_result_petr.append({
        "code": code,
        "recherche_count": info["recherche_count"],
        "thematiques_count": len(info["thematiques"]),
        "thematiques": list(info["thematiques"]),
        "type": "petr"
    })

filtered_result_petr = [
    item for item in final_result_petr
    if item["recherche_count"] >= 3 and item["thematiques_count"] >= 3
]
filtered_result_petr

[{'code': 'SM du Pays de la Déodatie',
  'recherche_count': 8,
  'thematiques_count': 3,
  'thematiques': ['Ressources_en_eau', 'Am%C3%A9nagement', 'Agriculture'],
  'type': 'petr'},
 {'code': "Syndicat des territoires de l'Est Cantal",
  'recherche_count': 7,
  'thematiques_count': 4,
  'thematiques': ['Inconfort_thermique',
   'Ressources_en_eau',
   'Biodiversit%C3%A9',
   'Agriculture'],
  'type': 'petr'},
 {'code': 'PETR du Pays de Gâtine',
  'recherche_count': 3,
  'thematiques_count': 3,
  'thematiques': ['Inconfort_thermique',
   'Biodiversit%C3%A9',
   'Gestion_des_risques'],
  'type': 'petr'}]

In [29]:
len(filtered_result_petr)

3

In [30]:
all_territories = filtered_result_commune + filtered_result_epci + filtered_result_pnr + filtered_result_ept + filtered_result_departement + filtered_result_petr

In [31]:
len(all_territories)

195

# Ajout du libellé des territoires


In [32]:
territoires = pd.read_csv("https://raw.githubusercontent.com/incubateur-ademe/facili-tacct/refs/heads/antoine/files/liste_searchbar.csv", sep=",")
lookup = dict(zip(territoires["search_code"], territoires["search_libelle"]))

In [33]:
for item in all_territories:
    code = item["code"]
    item["libelle"] = lookup.get(code, "Unknown")

In [34]:
len(all_territories)

195

# Résultats


In [35]:
df = pd.DataFrame(all_territories)
df.loc[df["type"] == "petr", "libelle"] = df.loc[df["type"] == "petr", "code"]
df.loc[df["type"] == "petr", "code"] = "unknown"
df

,code,recherche_count,thematiques_count,thematiques,type,libelle
0,01053,3,3,"[Ressources_en_eau, Inconfort_thermique, Gesti...",commune,Bourg-en-Bresse
1,17300,6,3,"[Inconfort_thermique, Ressources_en_eau, Gesti...",commune,La Rochelle
2,42168,6,4,"[Inconfort_thermique, Ressources_en_eau, Biodi...",commune,Pélussin
3,01004,28,5,"[Inconfort_thermique, Agriculture, Biodiversit...",commune,Ambérieu-en-Bugey
4,44109,10,5,"[Inconfort_thermique, Am%C3%A9nagement, Biodiv...",commune,Nantes
...,...,...,...,...,...,...
190,29,3,3,"[Inconfort_thermique, Am%C3%A9nagement, Biodiv...",departement,Finistère
191,56,4,4,"[Ressources_en_eau, Biodiversit%C3%A9, Agricul...",departement,Morbihan
192,unknown,8,3,"[Ressources_en_eau, Am%C3%A9nagement, Agricult...",petr,SM du Pays de la Déodatie
193,unknown,7,4,"[Inconfort_thermique, Ressources_en_eau, Biodi...",petr,Syndicat des territoires de l'Est Cantal


In [36]:
print(len(df.loc[df["type"] == "epci"]),
len(df.loc[df["type"] == "commune"]),
len(df.loc[df["type"] == "pnr"]),
len(df.loc[df["type"] == "petr"]),
len(df.loc[df["type"] == "departement"]),
len(df.loc[df["type"] == "ept"]))

135 25 4 3 28 0


In [37]:
# from google.colab import files
# output_path = f"north_star_{date}.xlsx"
# df.to_excel(output_path, index=False)
# files.download(output_path)

In [38]:
# df.to_csv(f'C:\\Antoine\\BetaGouv\\PostHog\\{date}\\north_star.csv', index=False)
# df.to_excel(f'C:\\Antoine\\BetaGouv\\PostHog\\{date}\\north_star.xlsx', index=False)